# Dota 2 Matchmaking: Data Cleaning and Preparation

---

## Dataset Collection & Overview

This dataset was collected by [**Devin Anzelmo**](https://www.kaggle.com/datasets/devinanzelmo/dota-2-matches/data) and contains 50,000 ranked ladder matches from the Dota 2 data dump created by [Opendota](https://www.opendota.com/). It was inspired by the [Dota 2 Matches](https://www.kaggle.com/jraramirez/dota-2-matches-dataset) data published by **Joe Ramir**. This is an updated and improved version of that dataset. The number of games in this dataset are played about every hour. 

> [**Quick look at how the dataset is structured**](https://www.kaggle.com/code/devinanzelmo/a-quick-look-at-dota-2-dataset)

|   CSV File             |  Description  | Notes |
|:-----------------------|:--------------|:------|
|  **Match Info**        |  |  |
| match                  | Top-level information about each match | `tower_status` and `barracks_status` are binary masks indicating whether various structures have been destroyed |
| players                | Statistics about player's individual performance in each match | Some players chose to hide their account_id and are marked as `0`. Player positions are enumerated from 0 to 4 for Radiant teams, while Dire teams go from 128 to 132 |
| player_time            | Contains XP, gold, and last-hit totals for each player at one-minute intervals | The suffix for each variable indicates the value of the `player_slot` variable |
| teamfights             | Basic information about each team fight | `start`, `end`, and `last_death` contain the time for those events in seconds |
| teamfights_players     | Detailed info about each team fight | Each row in `teamfights.csv` corresponds to ten rows in this file |
| chat                   | Chat log for all matches | These include the player's name in game |
| objectives             | Gives information on all the objectives completed, by which player and at what time |  |
| ability_upgrades       | Contains the upgrade performed at each level for each player |  |
| purchase_log           | Contains the time in seconds for each purchase made by every player in every match |  |
| **Game Info**          |  |  |
| ability_ids            | Ability names and ids | Use with `ability_upgrades.csv` to get the names of upgraded abilities |
| item_ids               | Contains `item_id` and item name | Use with `purchase_log.csv` to get the names of purchased items |
| hero_ids               | Contains the `name`, `hero_id`, and `localized_name` for each hero a player can pick | Concatenated this file with the one found [here](https://www.kaggle.com/datasets/nihalbarua/dota2-hero-preference-by-mmr) to obtain the `Primary Attribute` and possible Roles |
| cluster_region         | Contains the cluster number and geographic region | Allows to filter matches by region |
| patch_dates            | Release dates for various patches | Use `start_time` from `match.csv` to determine which patch was used to play in |
| **Historical Info**    |  |  |
| MMR                    | Contains `account_id` and players' **Matchmaking Rating** *(**MMR** for short)* | File extracted from the [**OpenDota Core Wiki**](https://github.com/odota/core/wiki/MMR-Data) where the original dataset is based from |
| player_ratings         | Skill data computed on **900k** previous matches and a possible way to measure skill rating when **MMR** is not available | `trueskill` ratings have two components, `mu`, which can be interpreted as the skill, with the higher value being better, and `sigma` which is the uncertainty of the rating. Negative `account_id` are players not appearing in other data available in this dataset |
| match_outcomes         | Results with `account_id` for **900k** matches occurring prior the rest of the dataset | Each match has data on two rows. the `rad` feature indicates whether the team is Radiant or Dire. *Useful for creating custom skill calculations* |
| **Tests**              |  |  |
| test_labels            | `match_id` and `radiant_win` as integer 1 or 0 |  |
| test_player            | Full player and match table with `hero_id`, `player_slot`, `match_id`, and `account_id`|  |

Despite going through the [**quick look into the dataset structure**](https://www.kaggle.com/code/devinanzelmo/a-quick-look-at-dota-2-dataset) posted above, I still have to make sure that the data is clean for EDA and modelling.

### Initial Setup

In [1]:
# Basic Data Science Libraries
import numpy as np # Linear algebra
import pandas as pd # Data processing

# System Libraries
import os # Miscellaneous operating system interfaces
import gc # Garbage collector interface
from subprocess import check_output # Saves results written to the current directory as output

# Removing the max columns limiter
pd.set_option('display.max_columns', None)

# Defining my clean and raw directory paths
clean_folder = '../Data/Clean'
raw_folder = '../Data/Raw'

# Printing files in my Raw Dataset
print(check_output(['ls', raw_folder]).decode('utf8'))

# Defining a function to read each file and learn its shape
dfs = {}
def read_file(file, **kwargs):
    '''
    This is a simple function to read a CSV file name as a string, assign it to a DataFrame and print its shape
    '''
    index = kwargs.get('index', None)
    
    assert isinstance(file, str), 'File name must be a string'
        
    # Include CSV files exclusively
    if file.endswith('.csv'):
            
        # Construct full file path
        file_path = os.path.join(raw_folder, file)
    
        # Assign to DataFrame
        key = file.split('.csv')[0] 
        dfs[key] = pd.read_csv(file_path, index_col=index)
        
        # Print the shape of the file
        return print(f'{file_path}:', '{:,} observations, {:,} features'.format(dfs[key].shape[0], dfs[key].shape[1]))

MMR.csv
ability_ids.csv
ability_upgrades.csv
chat.csv
cluster_regions.csv
hero_ids.csv
item_ids.csv
match.csv
match_outcomes.csv
objectives.csv
patch_dates.csv
player_ratings.csv
player_time.csv
players.csv
purchase_log.csv
teamfights.csv
teamfights_players.csv
test_labels.csv
test_player.csv
yasp_sample.json



---
## Match Data

In [42]:
# Loading up the MMR.csv.csv file
read_file('MMR.csv')

../Data/Raw/MMR.csv: 1,069,672 observations, 2 features


In [43]:
# Overview of the Patches DataFrame
dfs['MMR'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069672 entries, 0 to 1069671
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype
---  ------      --------------    -----
 0   account_id  1069672 non-null  int64
 1   MMR         1069672 non-null  int64
dtypes: int64(2)
memory usage: 16.3 MB


In [44]:
# Checking for null values
dfs['MMR'].isna().sum()

account_id    0
MMR           0
dtype: int64

In [39]:
# Looking at the DataFrame
dfs['player_ratings'].sample(20)

,total_wins,total_matches,trueskill_mu,trueskill_sigma
account_id,,,,
-55215220,6,9,30.285144,6.546324
-59445960,1,1,27.064935,8.095359
-172221859,2,2,30.030518,7.695559
-116517133,0,3,20.011843,7.694843
173290,4,13,19.746079,5.979182
78759,2,3,24.760290,7.787895
199862,4,6,26.520665,7.177555
-175871485,4,7,29.384085,6.845155
-279340236,0,2,22.828540,7.974001


In [41]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'mmr.csv')
dfs['MMR'].to_csv(file_path)

# Removing the file from memory
del dfs['MMR']
gc.collect()

0

### Chat

##### Data Formatting

In [ ]:
matches['chatlog'].sample(10)

I noticed that the chatlog data frame doesn't follow the conventions of the rest of the files for the Dire team positions (128 to 132) and doesn't include the account ID, but rather displays the account itself. If we want to extract the account name and merge it into other data frames that include the match ID and player slot, it would be best to create a new auxiliary column that combines the match ID and the player slot.

Additionally, the formatting of the column names doesn't match the rest of our dataframes. Let's change the formatting in the column names and the player slot values to match the rest of the dataset.

In [ ]:
# Renaming the columns
matches['chatlog'].rename(columns={'key': 'chat', 'slot': 'player_slot', 'unit': 'account'}, inplace=True)

# Changing the column order
matches['chatlog'] = matches['chatlog'][['match_id', 'player_slot', 'account', 'chat', 'time']]
matches['chatlog'].sample(10)

In [ ]:
# Checking the unique values in player_slot
print('Unique values:', matches['chatlog']['player_slot'].nunique())
np.sort(matches['chatlog']['player_slot'].unique())

In [ ]:
# Checking the rows with a player slot value = -9
matches['chatlog'].loc[matches['chatlog']['player_slot'] == -9, :]

Looks like there's only 4 observations with a `-9` player slot, and all of them have negative times indicating these messages were sent prior the start of the match. I believe that the `-9` value is for players who didn't pick a slot before the match started and were later assigned one.

In [ ]:
# Checking one of those match IDs with the same account 
matches['chatlog'].loc[(matches['chatlog']['match_id'] == 22654) & (matches['chatlog']['account'] == 'Dana'), :]

Just as suspected, the player's slot value changed once they chose a slot or the match started. I'll check the other values just to be sure and proceed to change them to their right value.

In [ ]:
# Replacing the value for Dana in match 22654
Dana_mask = (matches['chatlog']['match_id'] == 22654) & \
            (matches['chatlog']['account'] == 'Dana') & \
            (matches['chatlog']['player_slot'] == -9)

matches['chatlog'].loc[Dana_mask,'player_slot'] = 3
matches['chatlog'].loc[(matches['chatlog']['match_id'] == 22654) & (matches['chatlog']['account'] == 'Dana'), :]

In [ ]:
# Checking the player slot for VaiBy
matches['chatlog'].loc[(matches['chatlog']['match_id'] == 16613) & (matches['chatlog']['account'] == 'VaiBy <3 #Miracle Boi :D'), :]

In [ ]:
# Replacing the value for VaiBy in match 16613
VaiBy_mask = (matches['chatlog']['match_id'] == 16613) & \
            (matches['chatlog']['account'] == 'VaiBy <3 #Miracle Boi :D') & \
            (matches['chatlog']['player_slot'] == -9)

matches['chatlog'].loc[VaiBy_mask,'player_slot'] = 9
matches['chatlog'].loc[(matches['chatlog']['match_id'] == 16613) & (matches['chatlog']['account'] == 'VaiBy <3 #Miracle Boi :D'), :]

In [ ]:
# Checking the player slot for F@lcoN
matches['chatlog'].loc[(matches['chatlog']['match_id'] == 11215) & (matches['chatlog']['account'] == 'F@lcoN'), :]

In [ ]:
# Replacing the value for F@lcoN in match 11215
VaiBy_mask = (matches['chatlog']['match_id'] == 11215) & \
            (matches['chatlog']['account'] == 'F@lcoN') & \
            (matches['chatlog']['player_slot'] == -9)

matches['chatlog'].loc[VaiBy_mask,'player_slot'] = 6
matches['chatlog'].loc[(matches['chatlog']['match_id'] == 11215) & (matches['chatlog']['account'] == 'F@lcoN'), :]

In [ ]:
# Checking the player slot for Разрыватель пука
matches['chatlog'].loc[(matches['chatlog']['match_id'] == 8216) & (matches['chatlog']['account'] == 'Разрыватель пука'), :]

In [ ]:
# Replacing the value for Разрыватель пука in match 8216
VaiBy_mask = (matches['chatlog']['match_id'] == 8216) & \
            (matches['chatlog']['account'] == 'Разрыватель пука') & \
            (matches['chatlog']['player_slot'] == -9)

matches['chatlog'].loc[VaiBy_mask,'player_slot'] = 9
matches['chatlog'].loc[(matches['chatlog']['match_id'] == 8216) & (matches['chatlog']['account'] == 'Разрыватель пука'), :]

In [ ]:
# Checking the unique values in player_slot
print('Unique values:', matches['chatlog']['player_slot'].nunique())
np.sort(matches['chatlog']['player_slot'].unique())

Now that we only have 10 values, it's time to replace the values with the convention used in other dataframes:
- Slot 5 = Slot 128
- Slot 6 = Slot 129
- Slot 7 = Slot 130
- Slot 8 = Slot 131
- Slot 9 = Slot 132

In [ ]:
# Changing the player_slot to the general convention
slot_5 = matches['chatlog']['player_slot'] == 5
matches['chatlog'].loc[slot_5,'player_slot'] = 128

slot_6 = matches['chatlog']['player_slot'] == 6
matches['chatlog'].loc[slot_6,'player_slot'] = 129

slot_7 = matches['chatlog']['player_slot'] == 7
matches['chatlog'].loc[slot_7,'player_slot'] = 130

slot_8 = matches['chatlog']['player_slot'] == 8
matches['chatlog'].loc[slot_8,'player_slot'] = 131

slot_9 = matches['chatlog']['player_slot'] == 9
matches['chatlog'].loc[slot_9,'player_slot'] = 132

# Sanity Check
matches['chatlog'].sample(20)

Now our data frame is ready to create the new auxiliary column concatenating the match ID and player slot.

In [ ]:
# Creating a new column with the match_slot_id values
matches['chatlog'].insert(loc=2, column='match_slot_id',
                         value=matches['chatlog']['match_id'].astype(str)+'_'+matches['chatlog']['player_slot'].astype(str))

# Checking the new column
matches['chatlog'].sample(20)

Finally, we will add the match outcome to our table to give context to the players' mindsets behind their chat messages.

In [ ]:
# Obtaining the match outcomes
match_outcome = []
for i, row in matches['chatlog'].iterrows():
    if row['player_slot'] < 5:
        match_outcome.append((matches['overview'].iloc[row['match_id']]['radiant_win']).astype(int))
    else:
        match_outcome.append((~matches['overview'].iloc[row['match_id']]['radiant_win']).astype(int)) # Inverting the result for Dire players

matches['chatlog']['match_outcome'] = match_outcome
matches['chatlog'].sample(10)

##### Filling Missing Values

Now that we have worked out the formating and included our `match_slot_id` column, let's go ahead and explore the data frame.

In [ ]:
# Getting an overview of the updated data frame
matches['chatlog'].info()

In [ ]:
# Looking at our null values
matches['chatlog'].isna().sum()

Before removing any observations, I'll store the match_slot_ids and accounts in a dictionary to have as a reference.

In [ ]:
# Storing the account values and match_slot_ids in a dictionary
accounts = matches['chatlog'].groupby('match_slot_id')['account'].agg(lambda x: x.unique()[0])
accounts = accounts.reset_index()

ref['accounts'] = accounts.set_index('match_slot_id').to_dict()['account']
print('match_slot_id 9_131:', ref['accounts']['9_131']) # Sanity Check

In [ ]:
# Looking at the null values in chat
matches['chatlog'].loc[matches['chatlog']['chat'].isna(),:]

There's a possibility that these messages were censored in some way, so to avoid losing the observations, it's best to fill them with an empty string.

In [ ]:
# Filling these null values with an empty string ''
matches['chatlog'].fillna(value={'chat': ''}, inplace=True)
matches['chatlog'].loc[matches['chatlog']['chat'] == '',:]

I'll leave the null values on the accounts as they are for now, since they might be filled out when extracting the account IDs from other data frames.

In [ ]:
# Descriptive Statistics for time 
matches['chatlog'][['account', 'chat', 'time']].describe(include='all')

##### Visualizing the Distribution

In [ ]:
# Plotting the time distribution for chats
plt.figure()
fig = px.histogram(matches['chatlog']['time'],
                   title='Chat Messages Over Match Duration', 
                   text_auto=True, color_discrete_sequence=['indianred'], width=1020, height=480)
fig.update_layout(showlegend=False, xaxis_title='Time in seconds', yaxis_title='# Chat Messages')
fig.show()

Looking at the distribution, it's clear that the chat messages are heavily skewed to the right. Let's continue exploring the data and review similar messages written differently, such as the common expression **"gg"** *(derived from "good game")*.

In [ ]:
# Grouping the messages that contain gg
grouped_by_chat = matches['chatlog'].groupby(by=["chat"])[["match_id"]].count().reset_index()
grouped_by_chat.rename(columns={'match_id': 'occurrence'}, inplace=True)

print('Total times a message contained "gg": {:,}'.
      format(grouped_by_chat[grouped_by_chat['chat'].str.contains('gg')]['occurrence'].sum()), '\n')

print('Total times the message "gg" was sent: {:,}'.
     format(grouped_by_chat[grouped_by_chat['chat'] == 'gg']['occurrence'].sum()), '\n')

print('Total times " gg " was sent (i.e. not "ggez"): {:,}'.
     format(grouped_by_chat[grouped_by_chat['chat'].str.contains(fr'\b{"gg"}\b')]['occurrence'].sum()))

It's crucial to consider that the occurrences of these message variations vary significantly. This emphasizes the need to analyze multiple conditions to ensure accuracy by combining similar messages and aggregating those that appear more frequently in our data frame. Let's examine the frequency of the most common words in online gaming.

> **Note:** *For research purposes, I'll include profanity to identify positive and negative words used within gaming communities.*

In [ ]:
# Combining similar messages by checking multiple conditions

similar_chats = grouped_by_chat

# Common words used within the gaming community
keywords = ['gg', 'wp', 'glhf', 'lol', 'rofl', 'nice', 'afk', 'lag', 'report', 'rekt',
            'thanks', 'ez', 'noob', 'pog', 'kewk', 'kappa', 'asshole', 'fuck', 'bitch', 'fucker']

# Defining a function to check 
def check_for_keywords(row):
    output = ''
    for item in keywords:
        con0 = item != row['chat']
        con1 = bool(re.search(fr'\b{item.lower()} \w+', row['chat'].lower()))  # standalone words in front
        con2 = bool(re.search(fr'\w+ {item.lower()}\b', row['chat'].lower()))  # standalone words in back
        con3 = bool(re.search(fr'\w+ {item.lower()} \w+', row['chat'].lower()))  # standalone words in middle
        con4 = bool(re.search(fr'\b{item.lower()}\w+', row['chat'].lower()))  # in front of a word
        con5 = bool(re.search(fr'\w+ {item.lower()}[!=?-] \w+', row['chat'].lower()))  # word/number/symbol middle
        con6 = bool(re.search(fr'\b{item.lower()}[!=?-] \w+', row['chat'].lower()))  # word/number/symbol front
        con7 = bool(re.search(fr'\w+ {item.lower()}[!=?-]\b', row['chat'].lower()))  # word/number/symbol back
        
        # Switching row value with item if it matches with regex patterns
        if (con1 or con2 or con3 or con4 or con5 or con6 or con7) and con0:
            return item
        else:
            output = row['chat']
            
    return output

# Checking for the keywords in our similar_chats df
similar_chats['chat'] = similar_chats.apply(lambda row: check_for_keywords(row), axis=1)
agg_functions = {'chat': 'first', 'occurrence': 'sum'}

similar_chats = similar_chats.groupby('chat').agg(agg_functions).reset_index(drop=True).\
                sort_values('occurrence',ascending=False).reset_index(drop=True)

print('Keywords having two occurrences:', round(similar_chats[similar_chats['occurrence'] == 2].
                                                count()['chat'] / similar_chats.shape[0], 2))

It is safe to assume that most ocurrences happen only once, so we can focus our efforts on filtering out those instances.

In [ ]:
# Removing the chats with only one occurrence
two_or_more_chats = similar_chats[similar_chats['occurrence'] >= 2]

# Iterating for each row and appending it's value to a string
chat_messages = ''
for i, row in two_or_more_chats.iterrows():
    chat_messages += (row['chat'] + ' ') * row['occurrence']

# Plotting the word cloud
wordcloud = WordCloud(width=1020, height=480,
                      background_color='black',
                      colormap='Spectral',
                      stopwords=STOPWORDS,
                      collocations=False,
                      min_font_size = 10).generate(chat_messages)

plt.figure(figsize=(8,8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

##### Filtering by English Speaking Regions

In [ ]:
# Defining the regions of interest
eng_regions = {'US WEST', 'US EAST', 'EUROPE', 'AUSTRALIA'}

# Extracting the clusters and appending them to a list
eng_clusters = []

for region in eng_regions:
    for cluster in dota2['regions'][region]:
        eng_clusters.append(cluster)
    
eng_clusters = set(eng_clusters)

# Extracting the match IDs from those clusters
eng_match_ids = []

for i, match in enumerate(matches['overview']['match_id'][matches['overview']['cluster'].isin(eng_clusters)]):
    eng_match_ids.append(match)

eng_match_ids = set(eng_match_ids)

# Extracting the account IDs from those clusters
eng_player_ids = []

for i, player in enumerate(matches['players']['account_id'][matches['players']['match_id'].isin(eng_match_ids)]):
    eng_player_ids.append(player)

eng_player_ids = set(eng_player_ids)

# Printing the total unique values
print('Total matches in selected regions: {:,} ({})%'.format(len(eng_match_ids),
                                                            round((len(eng_match_ids) / matches['overview'].shape[0])*100, 2)))
print('Total English-speaking players: {:,} ({})%'.format(len(eng_player_ids), 
                                                          round((len(eng_player_ids) / 
                                                                 len(matches['players']['account_id'].unique()))*100, 2)))

In [ ]:
# Filtering out the chatlog by English-speaking regions
eng_chatlog = matches['chatlog'][matches['chatlog']['match_id'].isin(eng_match_ids)]
eng_chatlog

In [ ]:
print('Total English-speaking players in chatlog: {:,}'.format(eng_chatlog['account'].nunique()))

##### TF-IDF Vectorization

In [ ]:
# Test/Train Split
X = eng_chatlog['chat']
y = eng_chatlog['match_outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Defining a function to remove stopwords, punctuation, split sentences and lower case
stemmer = nltk.stem.PorterStemmer()
ENGLISH_STOP_WORDS = stopwords.words('english')

def my_tokenizer(sentence):
    # remove punctuation and set to lower case
    for punctuation_mark in string.punctuation:
        sentence = sentence.replace(punctuation_mark,'').lower()

    # split sentence into words
    listofwords = sentence.split(' ')
    listofstemmed_words = []
    
    # remove stopwords and any tokens that are just empty strings
    for word in listofwords:
        if (not word in ENGLISH_STOP_WORDS) and (word!=''):
            # Stem words
            stemmed_word = stemmer.stem(word)
            listofstemmed_words.append(stemmed_word)

    return listofstemmed_words

my_tokenizer('This is just a simple sanity check!! kekw pog ggwp')

### Players

This might be the data frame where most of the modelling will occur. So before moving forward and dropping any columns or rows, we have to create the `match_slot_id` column to merge the account names extracted from the chatlog.

In [ ]:
# Creating the new column with the new match_slot_id values
matches['players'].insert(loc=4, column='match_slot_id',
                         value=matches['players']['match_id'].astype(str)+'_'+matches['players']['player_slot'].astype(str))

# Checking the new column
matches['players'].sample(20)

In [ ]:
# Inserting the account names extracted from the chatlog
matches['players'].insert(2, 'account', matches['players']['match_slot_id'].map(ref['accounts']))
matches['players'].sample(20)

In [ ]:
# Looking at a small sample of the data frame 
matches['players'].sample(20)

In [ ]:
# Overview of the Positions data frame
matches['players'].info()

In [ ]:
# Getting the total null values per column
matches['players'].isna().sum().sort_values(ascending=False)\
[matches['players'].isna().sum().sort_values(ascending=False) >0]

The first thing that comes to mind is that we have a lot of features in this data frame. However, quite a few have almost no values in them, such as `unit_order_none`, `unit_order_taunt`, `unit_order_cast_rune`, `unit_order_patrol`, `unit_order_vector_target_position`, `unit_order_radar`, `unit_order_set_item_combine_lock` and `unit_order_continue`. Therefore it is safe to drop those columns entirely.

In [ ]:
# Dropping the columns with less than 10 rows with values
matches['players'].drop(columns=['unit_order_none','unit_order_taunt','unit_order_cast_rune','unit_order_patrol',
                                'unit_order_vector_target_position','unit_order_radar','unit_order_set_item_combine_lock',
                                'unit_order_continue'], inplace=True)

In [ ]:
# Getting the total null values per column
matches['players'].isna().sum().sort_values(ascending=False)\
[matches['players'].isna().sum().sort_values(ascending=False) >0]

Next, I'll check for any duplicated observations in our data.

In [ ]:
# Duplicated rows
print('Duplicated rows: {:,} ({})%'.format(matches['players'].duplicated().sum(),
                                           round((matches['players'].duplicated().sum()/matches['players'].shape[0])*100,2)))

Having confirmed we don't have any duplicated observations, we can go ahead and look into the missing values.

Apart from the columns added recently, all of our features in this data frame are integers and float values, which means we can examine the descriptive statistics of each one of them.

In [ ]:
# Descriptive Stats
matches['players'].describe()

I've also noticed in the descriptive statistics that the **25th percentile** of the players have an account ID of 0. As mentioned earlier, some players have opted to play anonymously, so their `account_id` is replaced with a value of `0`. Since accurately identifying each player and monitoring their in-game actions is crucial for our project, it's essential to remove all matches where we have one or more players with hidden `account_id` values.

In [ ]:
# Taking a look at the total hidden account IDs and null accounts
anonym_accounts = matches['players'][(matches['players']['account_id'] == 0) & \
                                    (matches['players']['account'].isna())]

print('Observations with anonymous ids: {:,}'.format(len(anonym_accounts)), 
      '({}%)'.format(round((len(anonym_accounts) / matches['players'].shape[0])*100, 2)))

In [ ]:
# Looking for the total number of matches that include one or more anonymous account ids
anonym_match_ids = anonym_accounts['match_id'].unique()

print('Total matches with at least one anonymous account: {:,}'.format(len(anonym_match_ids)))

Having so many matches with at least one player hiding their account ID makes it useless to isolate the matches with every account shown. However, since we are focused on each individual player, we'll go ahead and remove the players with a hidden account ID.

In [ ]:
# Filtering out the players with a hidden account id
matches['players'] = matches['players'][~((matches['players']['account_id'] == 0) & \
                                        (matches['players']['account'].isna()))]

print('Filtered dataframe shape:', matches['players'].shape)
matches['players'][matches['players']['match_id'] == 35791] # Checking a random match

In [ ]:
# Getting the total players we can identify
player_ids = matches['players']['account_id'].unique()
print('Total players in our dataset: {:,}'.format(len(player_ids)))

In [ ]:
# Checking how many matches were filtered out by having every player with a hidden ID
print('Remaining matches: {:,}'.format(len(matches['players']['match_id'].unique())))

Being able to get all 50,000 matches with at least one identifiable player is far better than expected.

In [ ]:
# Plotting the total real account ID distribution per match
plt.figure()
fig = px.histogram(matches['players'].groupby('match_id')[['account_id']].count(), nbins=10,
                   title='Distribution of identifiable players in matches', 
                   text_auto=True, color_discrete_sequence=['indianred'], width=1020, height=480)
fig.update_layout(showlegend=False, xaxis_title='Players identified in a Match', yaxis_title='# Matches')
fig.show()

### Overview

In [ ]:
# Taking a look at the features and values
matches['overview'].sample(10)

In [ ]:
matches['overview'].info()

In [ ]:
matches['overview'].describe()

### Time

In [ ]:
matches['time'][matches['time']['match_id'] == 49999]

### Objectives

In [ ]:
matches['objectives'].sample(10)

### Upgrades

In [ ]:
matches['upgrades'].sample(10)

### Purchases

In [ ]:
matches['purchases'].sample(10)

---

### Teamfights

#### Overview

In [ ]:
# Looking at a small sample of the data frame 
tfs['overview'].head(20)

Since multiple teamfights occur during a single match, it would be nice to know the order to get other significant stats.

In [ ]:
# Inserting the teamfight order from each match
tfs['overview'].insert(1, 'tf_order', tfs['overview'].groupby('match_id').cumcount().add(1))
tfs['overview'].head(20)

#### Breakdown

According to the notes from our files, it seems that for each observation in the `tfs['overview']` dataframe, we have a set of 10 *(one observation per player)* in the breakdown. If we end up merging the two tables together, it might come in handy to define a teamfight id.

In [ ]:
# First we need to know the total teamfights we have
print('Total teamfights: {:,}'.format(tfs['overview'].shape[0]))
print('Total observations in breakdown: {:,}'.format(tfs['breakdown'].shape[0]))
print('Teamfights have 10x observations:', tfs['overview'].shape[0] == (tfs['breakdown'].shape[0]/10))

In [ ]:
tfs['breakdown'].head(25)

In [ ]:
# Including the tf_id column in the detailed dataset
tfs['breakdown'].insert(1, 'tf_id', tfs['breakdown'].index // 10)
tfs['breakdown'].head(50)

#### Previous Outcomes

In [ ]:
ref['prev_outcomes'].sample(10)

#### Ratings

In [ ]:
ref['ratings'].sample(5)

In [ ]:
# This ranking formula penalizes players with fewer matches because there is more uncertainty
ref['ratings']['conservative_skill_estimate'] = ref['ratings']['trueskill_mu'] - 3*ref['ratings']['trueskill_sigma']

In [ ]:
ref['ratings'].sample(5)

#### MMR

Looking into the source of the dataframe, I discovered the [**OpenDota Core Repository**](https://github.com/odota/core/wiki/MMR-Data) from the original dump, where it was posted on their wiki page the file containing all the **Matchmaking Ratings** for all players sampled.

In [ ]:
ref['mmr'].sample(5)

---

### Tests

In [ ]:
test['matches'].sample(5)

In [ ]:
test['players'].sample(5)

---

## Game Metadata

In this section, we'll review and clean all the information and metadata about the game.

### Abilities

In [2]:
# Loading up the ability_ids.csv file
read_file('ability_ids.csv', index=0)

../Data/Raw/ability_ids.csv: 688 observations, 1 features


In [3]:
# Overview of the abilities dataframe
dfs['ability_ids'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 688 entries, 0 to 10002
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ability_name  688 non-null    object
dtypes: object(1)
memory usage: 10.8+ KB


































































































































































































We can see that there are no missing values in this dataframe. Let's look into the data itself.

In [4]:
# Looking at the first rows
dfs['ability_ids'].head()

,ability_name
ability_id,
0,ability_base
5001,default_attack
5002,attribute_bonus
5003,antimage_mana_break
5004,antimage_blink


Noticing that this file contains just the ability IDs and names. I'll go ahead and move it to my cleaned data folder.

In [5]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'ability_ids.csv')
dfs['ability_ids'].to_csv(file_path)

# Removing the file from memory
del dfs['ability_ids']
gc.collect()

0

### Items

In [6]:
# Loading up the item_ids.csv file
read_file('item_ids.csv', index=0)

../Data/Raw/item_ids.csv: 189 observations, 1 features


In [7]:
# Overview of the items dataframe
dfs['item_ids'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 189 entries, 1 to 1027
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_name  189 non-null    object
dtypes: object(1)
memory usage: 3.0+ KB


In [8]:
# Looking at the first rows
dfs['item_ids'].head()

,item_name
item_id,
1,blink
2,blades_of_attack
3,broadsword
4,chainmail
5,claymore


Same as the abilities, this file contains only the item names and IDs. I'll save it to the clean folder.

In [9]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'item_ids.csv')
dfs['item_ids'].to_csv(file_path)

# Removing the file from memory
del dfs['item_ids']
gc.collect()

0

### Heroes

In [10]:
# Loading up the hero_ids.csv file
read_file('hero_ids.csv')

../Data/Raw/hero_ids.csv: 112 observations, 5 features


In [11]:
# Overview of the Heroes DataFrame
dfs['hero_ids'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               112 non-null    object
 1   hero_id            112 non-null    int64 
 2   localized_name     112 non-null    object
 3   Primary Attribute  112 non-null    object
 4   Roles              112 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.5+ KB


In [12]:
# Looking at a sample of the DataFrame
dfs['hero_ids'].sample(10)

,name,hero_id,localized_name,Primary Attribute,Roles
75,npc_dota_hero_lycan,77,Lycan,all,"Carry, Pusher, Durable, Escape"
30,npc_dota_hero_riki,32,Riki,agi,"Carry, Escape, Disabler"
73,npc_dota_hero_silencer,75,Silencer,int,"Carry, Support, Disabler, Initiator, Nuker"
35,npc_dota_hero_warlock,37,Warlock,int,"Support, Initiator, Disabler"
74,npc_dota_hero_obsidian_destroyer,76,Outworld Destroyer,int,"Carry, Nuker, Disabler"
48,npc_dota_hero_dazzle,50,Dazzle,all,"Support, Nuker, Disabler"
63,npc_dota_hero_batrider,65,Batrider,all,"Initiator, Disabler, Escape"
18,npc_dota_hero_tiny,19,Tiny,str,"Carry, Nuker, Pusher, Initiator, Durable, Disa..."
66,npc_dota_hero_ancient_apparition,68,Ancient Apparition,int,"Support, Disabler, Nuker"
24,npc_dota_hero_lion,26,Lion,int,"Support, Disabler, Nuker, Initiator"


The roles appear to contain multiple traits that would make more sense if they were inside a set *(since I'm not interested in them having a particular order but don't want any duplicates)*. I have also observed that the `name` and `localized_name` fields contain similar values, so I will remove the `name` column and set the `hero_id` as its index.

In [13]:
# Setting the hero_id as the index
dfs['hero_ids'].set_index('hero_id', inplace=True)

# Dropping the name feature since it is redundant
dfs['hero_ids'].drop(columns='name', inplace=True)

# Renaming the localized_name to name and formatting Primary Attribute and Roles to follow the same as the rest
dfs['hero_ids'].rename(columns={'localized_name': 'name', 'Primary Attribute': 'primary_attribute', 'Roles': 'roles'},
                      inplace=True)

# Changing the Role values to list type
dfs['hero_ids']['roles'] = dfs['hero_ids']['roles'].apply(lambda x: set(x.split(', ')) if isinstance(x, str) else [])

# Looking at the first rows
dfs['hero_ids'].head()

,name,primary_attribute,roles
hero_id,,,
1,Anti-Mage,agi,"{Carry, Escape, Nuker}"
2,Axe,str,"{Disabler, Carry, Initiator, Durable}"
3,Bane,all,"{Disabler, Support, Durable, Nuker}"
4,Bloodseeker,agi,"{Disabler, Carry, Initiator, Nuker}"
5,Crystal Maiden,int,"{Disabler, Support, Nuker}"


In [14]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'hero_ids.csv')
dfs['hero_ids'].to_csv(file_path)

# Removing the file from memory
del dfs['hero_ids']
gc.collect()

0

### Regions

In [15]:
# Loading up the cluster_regions.csv file
read_file('cluster_regions.csv')

../Data/Raw/cluster_regions.csv: 53 observations, 2 features


In [16]:
# Overview of the Regions data frame
dfs['cluster_regions'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cluster  53 non-null     int64 
 1   region   53 non-null     object
dtypes: int64(1), object(1)
memory usage: 976.0+ bytes


In [17]:
# Looking at the data frame
dfs['cluster_regions']

,cluster,region
0,111,US WEST
1,112,US WEST
2,113,US WEST
3,121,US EAST
4,122,US EAST
5,123,US EAST
6,124,US EAST
7,131,EUROPE
8,132,EUROPE
9,133,EUROPE


This file only showcases the clusters *(which we can assume are game servers)* and their region. However, it might be more accessible if we grouped the clusters by region.

In [18]:
# Converting the dataframe into a dictionary with clusters as sets
dfs['cluster_regions'] = pd.DataFrame(dfs['cluster_regions'].groupby('region')['cluster'].apply(set))
dfs['cluster_regions']

,cluster
region,
AUSTRALIA,"{171, 172}"
AUSTRIA,"{192, 193, 191}"
BRAZIL,"{201, 202, 204}"
CHILE,"{241, 242}"
DUBAI,{161}
EUROPE,"{131, 132, 133, 134, 135, 136, 137, 138}"
INDIA,{261}
JAPAN,"{144, 145}"
PERU,{251}


In [19]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'regions.csv')
dfs['cluster_regions'].to_csv(file_path)

# Removing the file from memory
del dfs['cluster_regions']
gc.collect()

0

### Patches

In [20]:
# Loading up the patch_dates.csv file
read_file('patch_dates.csv')

../Data/Raw/patch_dates.csv: 19 observations, 2 features


In [21]:
# Overview of the Patches dataframe
dfs['patch_dates'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   patch_date  19 non-null     object 
 1   name        19 non-null     float64
dtypes: float64(1), object(1)
memory usage: 432.0+ bytes


In [22]:
# Looking at the dataframe
dfs['patch_dates']

,patch_date,name
0,2010-12-24T00:00:00Z,6.70
1,2011-01-21T00:00:00Z,6.71
2,2011-04-27T00:00:00Z,6.72
3,2011-12-24T00:00:00Z,6.73
4,2012-03-10T00:00:00Z,6.74
5,2012-09-30T00:00:00Z,6.75
6,2012-10-21T00:00:00Z,6.76
7,2012-12-15T00:00:00Z,6.77
8,2013-05-30T00:00:00Z,6.78
9,2013-11-24T00:00:00Z,6.79


Apart from changing the `patch_date` type, I'll change the index to the patch name and save it.

In [23]:
# Obtaining the original datatype
print('Orignial Data type:\n', dfs['patch_dates'].dtypes['patch_date'])

# Updating to DateTime type
dfs['patch_dates']['patch_date'] = pd.to_datetime(dfs['patch_dates']['patch_date'], yearfirst=True)
print('\nUpdated Data type:\n', dfs['patch_dates'].dtypes['patch_date'])

Orignial Data type:
 object

Updated Data type:
 datetime64[ns, UTC]


In [24]:
# Setting up the name as index
dfs['patch_dates'].set_index('name', inplace=True)
dfs['patch_dates']

,patch_date
name,
6.70,2010-12-24 00:00:00+00:00
6.71,2011-01-21 00:00:00+00:00
6.72,2011-04-27 00:00:00+00:00
6.73,2011-12-24 00:00:00+00:00
6.74,2012-03-10 00:00:00+00:00
6.75,2012-09-30 00:00:00+00:00
6.76,2012-10-21 00:00:00+00:00
6.77,2012-12-15 00:00:00+00:00
6.78,2013-05-30 00:00:00+00:00


In [25]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'patch_dates.csv')
dfs['patch_dates'].to_csv(file_path)

# Removing the file from memory
del dfs['patch_dates']
gc.collect()

0

### Positions

I've decided to create a new file to include the relationship between each player' position within the game and other relevant in-game data

In [26]:
positions = pd.DataFrame({
    'player_slot': [0,1,2,3,4, # Player positions for Radiant team
                    128,129,130,131,132], # Player positions for Dire team
    'side': ['Radiant','Radiant','Radiant','Radiant','Radiant', # Radiant team indicator
             'Dire','Dire','Dire','Dire','Dire'], # Dire team indicator
    'position': ['Carry', 'Midlaner', 'Offlaner', 'Roamer', 'Hard Support', # Position names for Radiant team
                 'Carry', 'Midlaner', 'Offlaner', 'Roamer', 'Hard Support'], # Position names for Dire team
    'roles': [{'Carry','Escape','Pushers'}, # Slot 0 roles
              {'Carry','Durable','Pushers','Disabler','Nuker'}, # Slot 1 roles
              {'Carry','Durable','Initiator','Pushers','Disabler'}, # Slot 2 roles
              {'Support','Escape','Initiator','Disabler'}, # Slot 3 roles
              {'Support','Escape','Initiator','Disabler'}, # Slot 4 roles
              {'Carry','Escape','Pushers'}, # Slot 128 roles
              {'Carry','Durable','Pushers','Disabler','Nuker'}, # Slot 129 roles
              {'Carry','Durable','Initiator','Pushers','Disabler'}, # Slot 130 roles
              {'Support','Escape','Initiator','Disabler'}, # Slot 131 roles
              {'Support','Escape','Initiator','Disabler'}], # Slot 132 roles
}).set_index('player_slot')

# Taking a look into the created dataframe
positions

,side,position,roles
player_slot,,,
0,Radiant,Carry,"{Carry, Pushers, Escape}"
1,Radiant,Midlaner,"{Disabler, Carry, Pushers, Durable, Nuker}"
2,Radiant,Offlaner,"{Disabler, Durable, Carry, Initiator, Pushers}"
3,Radiant,Roamer,"{Disabler, Support, Initiator, Escape}"
4,Radiant,Hard Support,"{Disabler, Support, Initiator, Escape}"
128,Dire,Carry,"{Carry, Pushers, Escape}"
129,Dire,Midlaner,"{Disabler, Carry, Pushers, Durable, Nuker}"
130,Dire,Offlaner,"{Disabler, Durable, Carry, Initiator, Pushers}"
131,Dire,Roamer,"{Disabler, Support, Initiator, Escape}"


In [27]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'positions.csv')
positions.to_csv(file_path)

# Removing the file from memory
del positions
gc.collect()

0

---
## Referencial Data

In this section, we'll explore the files containing historical and referential information.

### Previous Outcomes

In [28]:
# Loading up the match_outcomes.csv file
read_file('match_outcomes.csv')

../Data/Raw/match_outcomes.csv: 1,828,588 observations, 10 features


In [29]:
# Overview of the Patches dataframe
dfs['match_outcomes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828588 entries, 0 to 1828587
Data columns (total 10 columns):
 #   Column          Dtype
---  ------          -----
 0   match_id        int64
 1   account_id_0    int64
 2   account_id_1    int64
 3   account_id_2    int64
 4   account_id_3    int64
 5   account_id_4    int64
 6   start_time      int64
 7   parser_version  int64
 8   win             int64
 9   rad             int64
dtypes: int64(10)
memory usage: 139.5 MB


In [30]:
# Checking for null values
dfs['match_outcomes'].isna().sum()

match_id          0
account_id_0      0
account_id_1      0
account_id_2      0
account_id_3      0
account_id_4      0
start_time        0
parser_version    0
win               0
rad               0
dtype: int64

In [31]:
# Looking at the dataframe
dfs['match_outcomes'].head(20)

,match_id,account_id_0,account_id_1,account_id_2,account_id_3,account_id_4,start_time,parser_version,win,rad
0,1636204962,34549,0,0,-51743434,-120875154,1437014585,12,1,0
1,1636204962,0,61598,138825,0,207232,1437014585,12,0,1
2,1636322679,0,-44943233,-240360907,19599,0,1437019968,12,0,0
3,1636322679,-97530201,0,0,0,-116349387,1437019968,12,1,1
4,1637385965,0,0,0,104738,0,1437052551,12,1,0
5,1637385965,0,0,278620,278619,0,1437052551,12,0,1
6,1637623870,-123447796,68408,-100048908,-16784805,320715,1437058007,12,1,0
7,1637623870,-108454938,-251819996,0,51172,-106710926,1437058007,12,0,1
8,1637739731,320093,0,178850,-45490226,-119392638,1437060903,12,0,0
9,1637739731,0,241925,-115963827,14072,-67386586,1437060903,12,1,1


By the looks of it, it seems that we can use this file to train potential matchmaking models with previous match ups and outcomes. However, we can clean it up by renaming the columns to follow the same format as other files, as well as merging rows with the same match ID and assigning the **Dire** team account IDs to their respective position column.

In [32]:
# Checking the unique values from parser_version
dfs['match_outcomes']['parser_version'].unique()

array([12, 13, 14])

In [33]:
# Separating Radiant from Dire accounts
radiant = dfs['match_outcomes'][dfs['match_outcomes']['rad'] == 1].copy()
dire = dfs['match_outcomes'][dfs['match_outcomes']['rad'] == 0].copy()

# Renaming the columns from Dire
dire.rename(columns={
    'account_id_0': 'account_id_128',
    'account_id_1': 'account_id_129',
    'account_id_2': 'account_id_130',
    'account_id_3': 'account_id_131',
    'account_id_4': 'account_id_132',
}, inplace=True)

# Dropping unnecessary columns
dire.drop(columns=['start_time', 'parser_version', 'win', 'rad'], inplace=True)
radiant.drop(columns='rad', inplace=True)

# Merging both DataFrames
dfs['match_outcomes'] = pd.merge(radiant, dire, on='match_id')

# Organizing and renaming columns
dfs['match_outcomes'].set_index('match_id', inplace=True)
dfs['match_outcomes'].rename(columns={'win': 'radiant_win'}, inplace=True)
dfs['match_outcomes'] = dfs['match_outcomes'][[
    'start_time', 'account_id_0', 'account_id_1', 'account_id_2', 'account_id_3', 'account_id_4', 'radiant_win',
    'account_id_128', 'account_id_129', 'account_id_130', 'account_id_131', 'account_id_132', 'parser_version'
]]

# Looking at the new DataFrame
dfs['match_outcomes'].head(20)

,start_time,account_id_0,account_id_1,account_id_2,account_id_3,account_id_4,radiant_win,account_id_128,account_id_129,account_id_130,account_id_131,account_id_132,parser_version
match_id,,,,,,,,,,,,,
1636204962,1437014585,0,61598,138825,0,207232,0,34549,0,0,-51743434,-120875154,12
1636322679,1437019968,-97530201,0,0,0,-116349387,1,0,-44943233,-240360907,19599,0,12
1637385965,1437052551,0,0,278620,278619,0,0,0,0,0,104738,0,12
1637623870,1437058007,-108454938,-251819996,0,51172,-106710926,0,-123447796,68408,-100048908,-16784805,320715,12
1637739731,1437060903,0,241925,-115963827,14072,-67386586,1,320093,0,178850,-45490226,-119392638,12
1637843670,1437063709,179628,7317,72598,-54120646,0,0,-142035,-63684257,72554,213175,-45490226,12
1637995977,1437068379,0,-232879514,112015,0,85776,1,0,53936,-116357051,-130700702,-110612461,12
1638064585,1437070692,-167470448,0,305550,83093,-192152053,0,103832,-94111546,0,-113340211,-112424993,12
1638095252,1437071741,0,-87947497,248282,-65305327,0,1,0,0,0,0,0,12


In [34]:
# Shape of the new DataFrame
dfs['match_outcomes'].shape

(914294, 13)

In [35]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'prev_outcomes.csv')
dfs['match_outcomes'].to_csv(file_path)

# Removing the file from memory
del dfs['match_outcomes'], radiant, dire
gc.collect()

0

### Player Ratings

In [36]:
# Loading up the player_ratings.csv file
read_file('player_ratings.csv', index=0)

../Data/Raw/player_ratings.csv: 834,226 observations, 4 features


In [37]:
# Overview of the Patches dataframe
dfs['player_ratings'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 834226 entries, 236579 to 0
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   total_wins       834226 non-null  int64  
 1   total_matches    834226 non-null  int64  
 2   trueskill_mu     834226 non-null  float64
 3   trueskill_sigma  834226 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 31.8 MB


In [38]:
# Checking for null values
dfs['player_ratings'].isna().sum()

total_wins         0
total_matches      0
trueskill_mu       0
trueskill_sigma    0
dtype: int64

In [39]:
# Looking at the DataFrame
dfs['player_ratings'].sample(20)

,total_wins,total_matches,trueskill_mu,trueskill_sigma
account_id,,,,
-92922830,2,4,26.265667,7.396848
18640,5,15,21.498497,5.655581
-127574887,3,5,25.621482,7.278428
-207991367,2,4,23.367428,7.620751
-168721714,1,2,24.842361,7.842951
64785,2,4,26.171528,7.368785
-4499108,12,24,28.008783,4.821066
-68099320,1,1,27.299942,8.087977
27448,18,32,29.267251,4.620121


According to **Devin Anzelmo**, this file contains **True Skill** rating for players in the match and test data. It was calculated from this [**Kaggle Kernel**](https://www.kaggle.com/code/devinanzelmo/dota-2-skill-rating-with-trueskill), using the [**TrueSkill**](https://trueskill.org/) library.

He also suggests using the following ranking formula to penalyze players with fewer matches to compensate for uncertainty:
<center>$\mu_{\text{TrueSkill}}  - 3\sigma_{\text{TrueSkill}}$</center>

In [40]:
# Adding the penalyzed ranking
dfs['player_ratings']['conservative_skill_estimate'] = dfs['player_ratings']['trueskill_mu'] - \
                                                       3 * dfs['player_ratings']['trueskill_sigma']
dfs['player_ratings'].head(20)

,total_wins,total_matches,trueskill_mu,trueskill_sigma,conservative_skill_estimate
account_id,,,,,
236579,14,24,27.868035,5.212361,12.230953
-343,1,1,26.544163,8.065475,2.347736
-1217,1,1,26.521103,8.114989,2.176136
-1227,1,1,27.248025,8.092217,2.971375
-1284,0,1,22.931016,8.092224,-1.345657
308663,1,1,26.761476,8.108880,2.434837
79749,21,40,30.553417,3.868734,18.947214
-1985,0,1,23.263409,8.098020,-1.030652
-2160,8,12,27.426018,6.391300,8.252117


This new DataFrame has the potential of becoming a target variable for our matchmaking modelling. Let's save it with a more descriptive name on our clean directory.

In [41]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'trueskill.csv')
dfs['player_ratings'].to_csv(file_path)

# Removing the file from memory
del dfs['player_ratings']
gc.collect()

0

### MMR

In [42]:
# Loading up the MMR.csv file
read_file('MMR.csv', index=0)

../Data/Raw/MMR.csv: 1,069,672 observations, 1 features


In [43]:
# Overview of the Patches DataFrame
dfs['MMR'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1069672 entries, 103757918 to 93119769
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   MMR     1069672 non-null  int64
dtypes: int64(1)
memory usage: 16.3 MB


In [44]:
# Checking for null values
dfs['MMR'].isna().sum()

MMR    0
dtype: int64

In [45]:
# Looking at the DataFrame
dfs['MMR'].sample(20)

,MMR
account_id,
143834452,1782
166397030,3310
197325531,2914
143400491,2984
102132894,3328
48676066,4598
140373434,2860
124420876,3136
65610564,2109


Seems straight forward, however, there might be some slight diferences between ***True Skill*** and ***MMR***. I'll keep this file and eventually join both this and the trueskill file with the player data.

In [46]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'mmr.csv')
dfs['MMR'].to_csv(file_path)

# Removing the file from memory
del dfs['MMR']
gc.collect()

0

---
## Test Data

In this section, we'll explore the files starting with the label `test_`.

### Test Labels

In [49]:
# Loading up the test_labels.csv.csv file
read_file('test_labels.csv', index=0)

../Data/Raw/test_labels.csv: 100,000 observations, 1 features


In [50]:
# Overview of the Patches DataFrame
dfs['test_labels'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 50000 to 149999
Data columns (total 1 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   radiant_win  100000 non-null  int64
dtypes: int64(1)
memory usage: 1.5 MB


In [51]:
# Checking for null values
dfs['test_labels'].isna().sum()

radiant_win    0
dtype: int64

In [52]:
# Looking at the DataFrame
dfs['test_labels'].sample(20)

,radiant_win
match_id,
115055,0
98251,1
135984,0
94898,0
140557,1
95572,1
142643,0
138395,1
126398,0


I'll just rename this file to `test_outcomes` and move on.

In [53]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'test_outcomes.csv')
dfs['test_labels'].to_csv(file_path)

# Removing the file from memory
del dfs['test_labels']
gc.collect()

0

### Test Players

In [56]:
# Loading up the test_player.csv.csv file
read_file('test_player.csv')

../Data/Raw/test_player.csv: 1,000,000 observations, 4 features


In [57]:
# Overview of the Patches DataFrame
dfs['test_player'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype
---  ------       --------------    -----
 0   match_id     1000000 non-null  int64
 1   account_id   1000000 non-null  int64
 2   hero_id      1000000 non-null  int64
 3   player_slot  1000000 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [58]:
# Checking for null values
dfs['test_player'].isna().sum()

match_id       0
account_id     0
hero_id        0
player_slot    0
dtype: int64

In [60]:
# Looking at the DataFrame
dfs['test_player'].head(20)

,match_id,account_id,hero_id,player_slot
0,50000,117784,96,0
1,50000,158361,84,1
2,50000,158362,46,2
3,50000,137970,85,3
4,50000,1090,39,4
5,50000,2391,9,128
6,50000,2393,75,129
7,50000,2394,106,130
8,50000,36737,74,131
9,50000,2392,62,132


This DataFrame will be useful to run tests in the upcoming modelling stage. However, it might be useful to include the `match_slot_id` we created previously in the files above.

In [61]:
# Creating a new column with the match_slot_id values
dfs['test_player'].insert(loc=4, column='match_slot_id',
                         value=dfs['test_player']['match_id'].astype(str)+'_'+dfs['test_player']['player_slot'].astype(str))

# Checking the new column
dfs['test_player'].sample(20)

,match_id,account_id,hero_id,player_slot,match_slot_id
126811,62681,0,85,1,62681_1
719928,121992,7325,21,131,121992_131
72777,57277,0,71,130,57277_130
270022,77002,174547,23,2,77002_2
801806,130180,0,54,129,130180_129
105871,60587,0,100,1,60587_1
337525,83752,226667,45,128,83752_128
388871,88887,236840,33,1,88887_1
402730,90273,44715,93,0,90273_0
369080,86908,232467,73,0,86908_0


In [62]:
# Saving the file to the cleaned directory
file_path = os.path.join(clean_folder, 'test_players.csv')
dfs['test_player'].to_csv(file_path)

# Removing the file from memory
del dfs['test_player']
gc.collect()

0